In [1]:
# Imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow
import warnings


%matplotlib
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

Using TensorFlow backend.


Using matplotlib backend: <object object at 0x0000021A5AAA54C0>


In [2]:
%reload_ext watermark
%watermark -a "Gustavo M" --iversions

Author: Gustavo M

json       : 2.0.9
numpy      : 1.21.0
torch      : 1.12.0+cu116
tensorflow : 2.9.1
torchvision: 0.13.0
matplotlib : 3.5.1



In [5]:
# Checking if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA not available. Training will run on CPU')
else:
    print('CUDA available. Training will run on GPU.')

CUDA available. Training will run on GPU.


In [6]:
# Dataset loading

transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(10),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_data = datasets.CIFAR10('data', 
                                train = True,
                                download = True, 
                                transform = transform)

test_data = datasets.CIFAR10('data', 
                               train = False,
                               download = True, 
                               transform = transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data\cifar-10-python.tar.gz to data
Files already downloaded and verified


In [17]:
# Splitting data

num_train_samples = len(train_data)

indices = list(range(num_train_samples))
np.random.shuffle(indices)

valid_size = 0.2


split = int(np.floor(valid_size * num_train_samples))
idx_train, idx_valid = indices[split:], indices[:split]

train_samples = SubsetRandomSampler(idx_train)
valid_samples = SubsetRandomSampler(idx_valid)

In [19]:
# Data Loaders

num_workers = 0
batch_size = 20

loader_train = torch.utils.data.DataLoader(train_data, 
                                            batch_size = batch_size, 
                                            sampler = train_samples, 
                                            num_workers = num_workers)


loader_valid = torch.utils.data.DataLoader(train_data, 
                                           batch_size = batch_size, 
                                           sampler = valid_samples, 
                                           num_workers = num_workers)


loader_test = torch.utils.data.DataLoader(test_data, 
                                           batch_size = batch_size, 
                                           num_workers = num_workers)

In [20]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [21]:
# Convolutional Neural Network Model
class CNNmodel(nn.Module):
    
    def __init__(self):
        super(CNNmodel, self).__init__()
        
        # Input layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
        
        # Hidden Layer 
        self.conv2 = nn.Conv2d(16, 32, 3, padding = 1)
        
        # Hidden Layer 
        self.conv3 = nn.Conv2d(32, 64, 3, padding = 1)
        
        # Max Pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected 1
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        
        # Fully connected 2
        self.fc2 = nn.Linear(500, 10)
        
        # Dropout (Regularization)
        self.dropout = nn.Dropout(0.5)

    # Forward
    def forward(self, x):
        
        # Relu Activation
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        # Flattening
        x = x.view(-1, 64 * 4 * 4)
        
        # Add one dropout for regularization
        x = self.dropout(x)
        
        # Add first hidden layer with Relu activation
        x = F.relu(self.fc1(x))
        
        # Another dropout
        x = self.dropout(x)
        
        # Add second hidden layer with Relu activation
        x = self.fc2(x)
        return x

In [22]:
model = CNNmodel()

In [23]:
# Moving model to GPU if available
if train_on_gpu:
    model.cuda()

In [24]:
# Loss function cross-entropy
criterion = nn.CrossEntropyLoss()

In [25]:
# SGD
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [27]:
# Training

error_valid_min = np.Inf 
num_epochs = 30

for epoch in range(1, num_epochs + 1):

    # monitoring error on train and validation
    error_train = 0.0
    error_valid = 0.0
    
    # Inicia o treinamento do modelo
    model.train()
    
    # Loop batchs train data
    for batch_idx, (data, target) in enumerate(loader_train):
        
        # tensor to GPU
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        
        # Clearing gradients
        optimizer.zero_grad()
        
        # Forward
        output = model(data)
        
        # Calculating Loss
        loss = criterion(output, target)
        
        # Backward: Calculating Gradients
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        # Updating error
        error_train += loss.item() * data.size(0)
        
    # Model evaluation
    model.eval()
    
    # Loop batchs validation
    for batch_idx, (data, target) in enumerate(loader_valid):
        
        # tensors to GPU
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        
        # Forward
        output = model(data)
        
        # Calculating loss
        loss = criterion(output, target)
        
        # Updating error
        error_valid += loss.item() * data.size(0)
    
    # Average Error
    error_train = error_train / len(loader_train.dataset)
    error_valid = error_valid / len(loader_valid.dataset)
        
    # Print
    print('\nEpoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, 
                                                                                         error_train, 
                                                                                         error_valid))
    
    # Save the model whenever the validation loss decreases
    if error_valid <= error_valid_min:
        print('Validation Loss deacresed ({:.6f} --> {:.6f}). Saving model.'.format(error_valid_min,
                                                                                                 error_valid))
        torch.save(model.state_dict(), 'models/final_model.pt')
        error_valid_min = error_valid


Epoch: 1 	Training Loss: 1.465765 	Validation Loss: 0.325176
Validation Loss deacresed (inf --> 0.325176). Saving model.

Epoch: 2 	Training Loss: 1.305327 	Validation Loss: 0.297695
Validation Loss deacresed (0.325176 --> 0.297695). Saving model.

Epoch: 3 	Training Loss: 1.213470 	Validation Loss: 0.280039
Validation Loss deacresed (0.297695 --> 0.280039). Saving model.

Epoch: 4 	Training Loss: 1.151303 	Validation Loss: 0.265592
Validation Loss deacresed (0.280039 --> 0.265592). Saving model.

Epoch: 5 	Training Loss: 1.100394 	Validation Loss: 0.251384
Validation Loss deacresed (0.265592 --> 0.251384). Saving model.

Epoch: 6 	Training Loss: 1.063094 	Validation Loss: 0.243460
Validation Loss deacresed (0.251384 --> 0.243460). Saving model.

Epoch: 7 	Training Loss: 1.023097 	Validation Loss: 0.234593
Validation Loss deacresed (0.243460 --> 0.234593). Saving model.

Epoch: 8 	Training Loss: 0.996121 	Validation Loss: 0.225566
Validation Loss deacresed (0.234593 --> 0.225566). Sav

In [28]:
# Loading final model
model.load_state_dict(torch.load('models/final_model.pt'))

<All keys matched successfully>

In [30]:
# Evaluating model
error_test = 0.0

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()

# Loop batches test
for batch_idx, (data, target) in enumerate(loader_test):
    
    # tensors to GPU
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    
    # Forward
    output = model(data)
    
    # Calculating loss
    loss = criterion(output, target)
    
    # Updating Loss on Test
    error_test += loss.item() * data.size(0)
    
    # Propabilities to prediciton
    _, pred = torch.max(output, 1)    
    
    # Comparing predictions to real target value
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    
    # Calculating precision on each class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# Average loss
error_test = error_test / len(loader_test.dataset)
print('\nLoss on Test: {:.6f}\n'.format(error_test))

# Accuracy on each class
for i in range(10):
    if class_total[i] > 0:
        print('Test Acurracy on class %5s: %2d%% (%2d/%2d)' % (classes[i], 
                                                             100 * class_correct[i] / class_total[i],
                                                             np.sum(class_correct[i]), 
                                                             np.sum(class_total[i])))
    else:
        print('Test accuracy %5s:)' % (classes[i]))

# Calcula a acurácia total
print('\nTest Accuracy (Total): %2d%% (%2d/%2d)' % (100. * np.sum(class_correct) / np.sum(class_total),
                                                        np.sum(class_correct), 
                                                        np.sum(class_total)))


Loss on Test: 0.784673

Test Acurracy on class airplane: 74% (748/1000)
Test Acurracy on class automobile: 83% (834/1000)
Test Acurracy on class  bird: 50% (504/1000)
Test Acurracy on class   cat: 57% (578/1000)
Test Acurracy on class  deer: 70% (703/1000)
Test Acurracy on class   dog: 62% (629/1000)
Test Acurracy on class  frog: 83% (837/1000)
Test Acurracy on class horse: 79% (797/1000)
Test Acurracy on class  ship: 82% (828/1000)
Test Acurracy on class truck: 83% (834/1000)

Test Accuracy (Total): 72% (7292/10000)
